In [8]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely as sp

In [9]:
ca = gpd.read_file('preprocessing/ca-state-boundary')
ca = ca.to_crs(4326) # reproject to WGS84 lon lat

## Generate Grid
Takes around 1 minute to run for GRID_RESOLUTION = 200

In [10]:
GRID_RESOLUTION = 100  # this ** 2 = num of generated points

# generate a uniform grid of points based within california's bounding box
lon_min, lat_min, lon_max, lat_max = ca.total_bounds
lons = np.linspace(lon_min, lon_max, GRID_RESOLUTION)
lats = np.linspace(lat_min, lat_max, GRID_RESOLUTION)
pts = np.dstack(np.meshgrid(lons, lats)).reshape(-1, 2)
grid = gpd.GeoSeries(gpd.points_from_xy(pts[:, 0], pts[:, 1]))
grid = grid[grid.within(ca.unary_union)]

## Save Coordinates to File

In [11]:
grid_df = pd.DataFrame([[p.xy[0][0], p.xy[1][0]] for p in grid.geometry])
grid_df.columns = ['lon', 'lat']
grid_df.to_csv('deploy/grid.csv')